In [13]:
import numpy as np
import pandas as pd
import json

# For each region, create a list of its GID

## Load the IVS data grouped by region

In [14]:
IVS_region = pd.read_csv("out/IVS_grouped_by_country_region.csv", encoding="utf-16", low_memory=False)
IVS_region.head()

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Region Numeric,Region Name,NUTS1,NUTS2,NUTS3,...,X053_1,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10
0,EVS,1981-1984,56,Belgium,BEL,56021,BE: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,EVS,1981-1984,56,Belgium,BEL,56022,BE: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,EVS,1981-1984,56,Belgium,BEL,56023,BE: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,EVS,1981-1984,56,Belgium,BEL,56024,BE: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,EVS,1981-1984,56,Belgium,BEL,56025,BE: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Load the region codes to GID dictionaries and add the list of GID to the IVS data

## WVS/EVS

In [15]:
VWS_to_gid = pd.read_csv("gid/region_codes_to_GID_WVS_EVS.csv", low_memory=False)
VWS_to_gid.drop("Region Name", inplace=True, axis=1)
#remove empty lines
VWS_to_gid.dropna(how="all", inplace=True) 
VWS_to_gid['Country Numeric'] = VWS_to_gid['Country Numeric'].astype(np.int64)
VWS_to_gid['Region Numeric'] = VWS_to_gid['Region Numeric'].astype(np.int64)
VWS_to_gid

,Survey,Wave,Country Numeric,Country ISO3,Region Numeric,gid
0,WVS,2010-2014,840,USA,840001,USA.20_1
1,WVS,2010-2014,840,USA,840001,USA.46_1
2,WVS,2010-2014,840,USA,840001,USA.30_1
3,WVS,2010-2014,840,USA,840001,USA.22_1
4,WVS,2010-2014,840,USA,840001,USA.40_1
...,...,...,...,...,...,...
818,WVS,2010-2014,752,SWE,752020,SWE.17_1
819,WVS,2010-2014,752,SWE,752026,SWE.18_1
820,WVS,2010-2014,752,SWE,752024,SWE.19_1
821,WVS,2010-2014,752,SWE,752021,SWE.20_1


In [16]:
#VWS_to_list_gid = VWS_to_gid.groupby(list(VWS_to_gid.columns[:-1])).agg({'gid':lambda x: ':'.join(list(x))}).reset_index()

VWS_to_list_gid = VWS_to_gid.groupby(list(VWS_to_gid.columns[:-1])).agg({'gid':lambda x: list(x)}).reset_index()

VWS_to_list_gid

,Survey,Wave,Country Numeric,Country ISO3,Region Numeric,gid
0,WVS,1999-2004,50,BGD,50001,[BGD.3.1_1]
1,WVS,1999-2004,50,BGD,50002,[BGD.2.4_1]
2,WVS,1999-2004,50,BGD,50003,[BGD.1.2_1]
3,WVS,1999-2004,50,BGD,50004,[BGD.3.10_1]
4,WVS,1999-2004,50,BGD,50005,[BGD.3.17_1]
...,...,...,...,...,...,...
367,WVS,2010-2014,840,USA,840005,"[USA.4_1, USA.19_1, USA.37_1, USA.44_1]"
368,WVS,2010-2014,840,USA,840006,"[USA.14_1, USA.15_1, USA.23_1, USA.36_1, USA.5..."
369,WVS,2010-2014,840,USA,840007,"[USA.16_1, USA.17_1, USA.24_1, USA.26_1, USA.2..."
370,WVS,2010-2014,840,USA,840008,"[USA.3_1, USA.6_1, USA.13_1, USA.27_1, USA.29_..."


In [17]:
len(VWS_to_list_gid["Region Numeric"].unique())

372

In [18]:
IVS_subset_VWS = pd.merge(IVS_region, VWS_to_list_gid, 
                on = ['Survey', 'Wave', 'Country Numeric', 'Country ISO3',
                                    'Region Numeric'],
                sort=False)

In [19]:
IVS_subset_VWS.head()

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Region Numeric,Region Name,NUTS1,NUTS2,NUTS3,...,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10,gid
0,WVS,1999-2004,50,Bangladesh,BGD,50001,BD: Dhaka,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BGD.3.1_1]
1,WVS,1999-2004,50,Bangladesh,BGD,50002,BD: Chittagong,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BGD.2.4_1]
2,WVS,1999-2004,50,Bangladesh,BGD,50003,BD: Barisal,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BGD.1.2_1]
3,WVS,1999-2004,50,Bangladesh,BGD,50004,BD: Mymensingh,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BGD.3.10_1]
4,WVS,1999-2004,50,Bangladesh,BGD,50005,BD: Tangail,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BGD.3.17_1]


In [20]:
# lets check if we left something out
VWS_to_list_gid[~VWS_to_list_gid["Region Numeric"].isin(IVS_subset_VWS["Region Numeric"])]

,Survey,Wave,Country Numeric,Country ISO3,Region Numeric,gid


## NUTS

In [21]:
NUTS_to_gid = pd.read_csv("gid/region_codes_to_GID_NUTS.csv", low_memory=False)
NUTS_to_gid.drop(["Region Name", "Region Numeric", "NUTS3"], inplace=True, axis=1)
#remove empty lines
NUTS_to_gid.dropna(how="all", inplace=True) 
NUTS_to_gid['Country Numeric'] = NUTS_to_gid['Country Numeric'].astype(np.int64)

NUTS_to_gid

,Survey,Wave,Country Numeric,Country ISO3,NUTS1,NUTS2,gid
0,EVS,2008-2010,372,IRL,IE0,-1,IRL
1,EVS,2008-2010,372,IRL,IE0,IE01,IRL.16_1
2,EVS,2008-2010,372,IRL,IE0,IE01,IRL.20_1
3,EVS,2008-2010,372,IRL,IE0,IE01,IRL.15_1
4,EVS,2008-2010,372,IRL,IE0,IE01,IRL.11_1
...,...,...,...,...,...,...,...
401,EVS,2008-2010,826,GBR,GB-GBNM,GB-GBNM6,GBR.3.17_1
402,EVS,2008-2010,826,GBR,GB-GBNM,GB-GBNM6,GBR.3.20_1
403,EVS,2008-2010,826,GBR,GB-GBNM,GB-GBNM6,GBR.3.21_1
404,EVS,2008-2010,826,GBR,GB-GBNM,GB-GBNM6,GBR.3.23_1


In [22]:
len(NUTS_to_gid.NUTS2.unique())

103

In [23]:
#NUTS_to_list_gid = NUTS_to_gid.groupby(list(NUTS_to_gid.columns[:-1])).agg({'gid':lambda x: ':'.join(list(x))}).reset_index()

NUTS_to_list_gid = NUTS_to_gid.groupby(list(NUTS_to_gid.columns[:-1])).agg({'gid':lambda x: list(x)}).reset_index()


NUTS_to_list_gid

,Survey,Wave,Country Numeric,Country ISO3,NUTS1,NUTS2,gid
0,EVS,2008-2010,56,BEL,BE1,BE10,[BEL.1.1_1]
1,EVS,2008-2010,56,BEL,BE2,BE21,[BEL.2.1_1]
2,EVS,2008-2010,56,BEL,BE2,BE22,[BEL.2.2_1]
3,EVS,2008-2010,56,BEL,BE2,BE23,[BEL.2.3_1]
4,EVS,2008-2010,56,BEL,BE2,BE24,[BEL.2.4_1]
...,...,...,...,...,...,...,...
98,EVS,2008-2010,826,GBR,GB-GBNL,GB-GBNL2,"[GBR.4.13_1, GBR.4.15_1, GBR.4.5_1, GBR.4.21_1..."
99,EVS,2008-2010,826,GBR,GB-GBNM,GB-GBNM2,"[GBR.3.3_1, GBR.3.5_1, GBR.3.7_1, GBR.3.10_1, ..."
100,EVS,2008-2010,826,GBR,GB-GBNM,GB-GBNM3,"[GBR.3.4_1, GBR.3.6_1, GBR.3.8_1, GBR.3.9_1, G..."
101,EVS,2008-2010,826,GBR,GB-GBNM,GB-GBNM6,"[GBR.3.4_1, GBR.3.13_1, GBR.3.17_1, GBR.3.20_1..."


In [24]:
IVS_subset_NUTS = pd.merge(IVS_region, NUTS_to_list_gid, 
                on = ['Survey', 'Wave', 'Country Numeric', 'Country ISO3',
                      'NUTS1', 'NUTS2'],
                sort=False)

In [25]:
IVS_subset_NUTS

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Region Numeric,Region Name,NUTS1,NUTS2,NUTS3,...,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10,gid
0,EVS,2008-2010,56,Belgium,BEL,-5,Unknown,BE1,BE10,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BEL.1.1_1]
1,EVS,2008-2010,56,Belgium,BEL,-5,Unknown,BE2,BE21,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BEL.2.1_1]
2,EVS,2008-2010,56,Belgium,BEL,-5,Unknown,BE2,BE22,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BEL.2.2_1]
3,EVS,2008-2010,56,Belgium,BEL,-5,Unknown,BE2,BE23,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BEL.2.3_1]
4,EVS,2008-2010,56,Belgium,BEL,-5,Unknown,BE2,BE24,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BEL.2.4_1]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,EVS,2008-2010,826,United Kingdom of Great Britain and Northern I...,GBR,-5,Unknown,GB-GBNL,GB-GBNL2,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[GBR.4.13_1, GBR.4.15_1, GBR.4.5_1, GBR.4.21_1..."
99,EVS,2008-2010,826,United Kingdom of Great Britain and Northern I...,GBR,-5,Unknown,GB-GBNM,GB-GBNM2,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[GBR.3.3_1, GBR.3.5_1, GBR.3.7_1, GBR.3.10_1, ..."
100,EVS,2008-2010,826,United Kingdom of Great Britain and Northern I...,GBR,-5,Unknown,GB-GBNM,GB-GBNM3,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[GBR.3.4_1, GBR.3.6_1, GBR.3.8_1, GBR.3.9_1, G..."
101,EVS,2008-2010,826,United Kingdom of Great Britain and Northern I...,GBR,-5,Unknown,GB-GBNM,GB-GBNM6,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[GBR.3.4_1, GBR.3.13_1, GBR.3.17_1, GBR.3.20_1..."


## Concatenate the two dataframes

In [26]:
IVS_region = pd.concat([IVS_subset_VWS, IVS_subset_NUTS], ignore_index=True)
IVS_region

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Region Numeric,Region Name,NUTS1,NUTS2,NUTS3,...,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10,gid
0,WVS,1999-2004,50,Bangladesh,BGD,50001,BD: Dhaka,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BGD.3.1_1]
1,WVS,1999-2004,50,Bangladesh,BGD,50002,BD: Chittagong,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BGD.2.4_1]
2,WVS,1999-2004,50,Bangladesh,BGD,50003,BD: Barisal,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BGD.1.2_1]
3,WVS,1999-2004,50,Bangladesh,BGD,50004,BD: Mymensingh,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BGD.3.10_1]
4,WVS,1999-2004,50,Bangladesh,BGD,50005,BD: Tangail,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BGD.3.17_1]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,EVS,2008-2010,826,United Kingdom of Great Britain and Northern I...,GBR,-5,Unknown,GB-GBNL,GB-GBNL2,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[GBR.4.13_1, GBR.4.15_1, GBR.4.5_1, GBR.4.21_1..."
471,EVS,2008-2010,826,United Kingdom of Great Britain and Northern I...,GBR,-5,Unknown,GB-GBNM,GB-GBNM2,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[GBR.3.3_1, GBR.3.5_1, GBR.3.7_1, GBR.3.10_1, ..."
472,EVS,2008-2010,826,United Kingdom of Great Britain and Northern I...,GBR,-5,Unknown,GB-GBNM,GB-GBNM3,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[GBR.3.4_1, GBR.3.6_1, GBR.3.8_1, GBR.3.9_1, G..."
473,EVS,2008-2010,826,United Kingdom of Great Britain and Northern I...,GBR,-5,Unknown,GB-GBNM,GB-GBNM6,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[GBR.3.4_1, GBR.3.13_1, GBR.3.17_1, GBR.3.20_1..."


# Create dictionary with the anwers descriptions

In [27]:
IVS_variables = pd.read_csv("input/IVS_Variable_List.csv")

In [28]:
IVS_variables

,Included,Name,Label,Last EVS,Last WVS,Categories
0,1,A001,Important in life: Family,EVS 2008,WVS6 (2010-2012),1:Very important\n2:Rather important\n3:Not ve...
1,1,A002,Important in life: Friends,EVS 2008,WVS6 (2010-2012),1:Very important\n2:Rather important\n3:Not ve...
2,0,A003,Important in life: Leisure time,EVS 2008,WVS6 (2010-2012),-5:Missing; Unknown\n-4:Not asked in survey\n-...
3,0,A004,Important in life: Politics,EVS 2008,WVS6 (2010-2012),1:Very important\n2:Rather important\n3:Not ve...
4,1,A005,Important in life: Work,EVS 2008,WVS6 (2010-2012),1:Very important\n2:Rather important\n3:Not ve...
...,...,...,...,...,...,...
1422,0,w007,Spouse/partner had/has how many employees,EVS 2008,NaN,1:None\n2:1-9\n3:10-24\n4:25 or more\n-5:Missi...
1423,0,w008,Does spouse/partner supervise someone,EVS 2008,NaN,0:No\n1:Yes\n-5:Missing; Unknown\n-4:Not asked...
1424,0,w009,How many people does she/he supervise,EVS 2008,NaN,1:1-9\n2:10-24\n3:25 or more\n-5:Missing; Unkn...
1425,0,w010,Spouse/partner experienced unemployment longer...,EVS 2008,NaN,0:No\n1:Yes\n-5:Missing; Unknown\n-4:Not asked...


In [29]:
IVS_variables = IVS_variables.loc[IVS_variables.Included == 1].copy()
IVS_variables.drop(["Included", "Last EVS", "Last WVS"], axis=1, inplace=True)

In [30]:
cat_dict_column = []

for idx, row in IVS_variables.iterrows():
    codes = {}
    for line in row.Categories.split("\n")[:-1]: # there is always a \n at the end
        try:
            key = line.split(":")[0]
            value = line.split(":")[1]
            codes[key] = value
        except:
            continue
    cat_dict_column.append(codes)

In [31]:
IVS_variables["Categories"] = cat_dict_column

In [32]:
IVS_variables.set_index("Name", inplace=True)

In [33]:
variables_dict = IVS_variables.to_dict(orient="index")

## We need to pop some keys
ZMEN_010 is not present in the dataset

the following ones are the values we aggregated on

S002
S002EVS
S001
S017

X048WVS
X048
x048a_n1
x048b_n2
x048c_n3

In [34]:
len(variables_dict.keys())

87

In [35]:
to_drop = ["ZMEN_010", "S002", "S002EVS", "S001", "S017",
          "X048WVS", "X048", "x048a_n1", "x048b_n2", "x048c_n3"]


for key in to_drop:
    variables_dict.pop(key)

len(variables_dict.keys())

77

# Dump the answers in a json

In [36]:
IVS_region['json'] = IVS_region.apply(lambda x: x.to_dict(), axis=1)

In [37]:
# now we have a flat dict, we will need to organize it
#IVS_region.json[0]

In [38]:
properties = []

for idx, row in IVS_region.iterrows():
    new_dict = {}
    
    # for each row we create a dict
    # {..., 
    #  column_id: {label: 'description',
    #              frequencies: {dict with the anwers onehot encoded}
    #.             categories: {dict with the answers codes}}, 
    #  ...}
    
    # label and categories are from variables_dict
    
    # add the original region labels
    new_dict["original region code"]= {
        'Region Numeric': row.json['Region Numeric'],
        'Region Name': row.json['Region Name'],
        'NUTS1': row.json['NUTS1'],
        'NUTS2': row.json['NUTS2'],
        'NUTS3': row.json['NUTS3'],
    }
    
    
    for answer_code in variables_dict.keys():
        
        # we already have label and categories
        sub_dict = variables_dict[answer_code]
        
        # we need frequencies
        # in the row json we have a flat dict
        # we want to aggregate by name of the answer
        sub_dict["Frequencies"] = {}
        
        for key, item in row.json.items():
            if key.startswith(answer_code):
                sub_dict["Frequencies"][key] = item
        new_dict[answer_code] = sub_dict
        
    properties.append(new_dict)    

In [39]:
df_to_save = IVS_region[['Survey', 'Wave', 'Country Numeric', 
                           'Country Name', 'Country ISO3', 'Sample Size', 'gid']].copy()
df_to_save["Properties"] = properties

In [40]:
df_to_save

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Sample Size,gid,Properties
0,WVS,1999-2004,50,Bangladesh,BGD,199,[BGD.3.1_1],{'original region code': {'Region Numeric': 50...
1,WVS,1999-2004,50,Bangladesh,BGD,121,[BGD.2.4_1],{'original region code': {'Region Numeric': 50...
2,WVS,1999-2004,50,Bangladesh,BGD,105,[BGD.1.2_1],{'original region code': {'Region Numeric': 50...
3,WVS,1999-2004,50,Bangladesh,BGD,128,[BGD.3.10_1],{'original region code': {'Region Numeric': 50...
4,WVS,1999-2004,50,Bangladesh,BGD,81,[BGD.3.17_1],{'original region code': {'Region Numeric': 50...
...,...,...,...,...,...,...,...,...
470,EVS,2008-2010,826,United Kingdom of Great Britain and Northern I...,GBR,22,"[GBR.4.13_1, GBR.4.15_1, GBR.4.5_1, GBR.4.21_1...",{'original region code': {'Region Numeric': -5...
471,EVS,2008-2010,826,United Kingdom of Great Britain and Northern I...,GBR,43,"[GBR.3.3_1, GBR.3.5_1, GBR.3.7_1, GBR.3.10_1, ...",{'original region code': {'Region Numeric': -5...
472,EVS,2008-2010,826,United Kingdom of Great Britain and Northern I...,GBR,43,"[GBR.3.4_1, GBR.3.6_1, GBR.3.8_1, GBR.3.9_1, G...",{'original region code': {'Region Numeric': -5...
473,EVS,2008-2010,826,United Kingdom of Great Britain and Northern I...,GBR,6,"[GBR.3.4_1, GBR.3.13_1, GBR.3.17_1, GBR.3.20_1...",{'original region code': {'Region Numeric': -5...


In [42]:
df_to_save.to_pickle("to_upload/IVS_region_GID.pkl")

# UPLOAD
## Loads the data from csv, there is no need to run the cells above

In [44]:
import pandas as pd
import psycopg2
import json

In [45]:
def create_IVS_region_table(cur):
    sql_create_country_stats_table = """ 
        CREATE TABLE IF NOT EXISTS IVS_region (
            survey text NOT NULL,
            wave text NOT NULL,
            country text NOT NULL,
            countrycode text NOT NULL,
            sample_size integer NOT NULL,
            gid text[] NOT NULL,
            properties jsonb,
            UNIQUE (survey, wave, country, countrycode, gid),
            PRIMARY KEY (survey, wave, country, countrycode, gid)
        )"""

    cur.execute(sql_create_country_stats_table)

    print("IVS_country table created")

In [46]:
def drop_IVS_region_table(cur):
    sql_drop_IVS_region_table = """DROP TABLE IVS_region"""
    
    try:
        cur.execute(sql_drop_IVS_region_table)
        conn.commit() 
        print("IVS_region table deleted")
    except:
        print("IVS_region not present")

In [47]:
def import_IVS_region_data(cur,data):
    
    INSERT_STATEMENT = 'INSERT INTO IVS_region \
                            (survey, wave, \
                             country, countrycode,  \
                             sample_size, gid, \
                             properties) \
                        VALUES (%s, %s, \
                                %s, %s, \
                                %s, %s, \
                                %s);'

    for idx, row in data.iterrows():
        cur.execute(INSERT_STATEMENT, (row['Survey'], row['Wave'], 
                                       row['Country Name'], row['Country ISO3'], 
                                       row['Sample Size'], row['gid'],
                                       json.dumps(row["Properties"])))

In [48]:
# Connect to covid19db.org
conn = psycopg2.connect(
    host='covid19db.org',
    port=5432,
    dbname='covid19db_adm_play',
    user='covid19db_adm_rw',
    password='fGt962FdeG2yXj3c4d3'
)

cur = conn.cursor()

In [49]:
drop_IVS_region_table(cur)
conn.commit()


create_IVS_region_table(cur)
conn.commit()


#load the table
data = pd.read_pickle("to_upload/IVS_region_GID.pkl") 

import_IVS_region_data(cur, data)
conn.commit()
print("data uploaded")


IVS_region table deleted
IVS_country table created
data uploaded


In [50]:
cur.execute("""SELECT * FROM IVS_region""")

data = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
data


,survey,wave,country,countrycode,sample_size,gid,properties
0,WVS,1999-2004,Bangladesh,BGD,199,[BGD.3.1_1],{'A001': {'Label': 'Important in life: Family'...
1,WVS,1999-2004,Bangladesh,BGD,121,[BGD.2.4_1],{'A001': {'Label': 'Important in life: Family'...
2,WVS,1999-2004,Bangladesh,BGD,105,[BGD.1.2_1],{'A001': {'Label': 'Important in life: Family'...
3,WVS,1999-2004,Bangladesh,BGD,128,[BGD.3.10_1],{'A001': {'Label': 'Important in life: Family'...
4,WVS,1999-2004,Bangladesh,BGD,81,[BGD.3.17_1],{'A001': {'Label': 'Important in life: Family'...
...,...,...,...,...,...,...,...
470,EVS,2008-2010,United Kingdom of Great Britain and Northern I...,GBR,22,"[GBR.4.13_1, GBR.4.15_1, GBR.4.5_1, GBR.4.21_1...",{'A001': {'Label': 'Important in life: Family'...
471,EVS,2008-2010,United Kingdom of Great Britain and Northern I...,GBR,43,"[GBR.3.3_1, GBR.3.5_1, GBR.3.7_1, GBR.3.10_1, ...",{'A001': {'Label': 'Important in life: Family'...
472,EVS,2008-2010,United Kingdom of Great Britain and Northern I...,GBR,43,"[GBR.3.4_1, GBR.3.6_1, GBR.3.8_1, GBR.3.9_1, G...",{'A001': {'Label': 'Important in life: Family'...
473,EVS,2008-2010,United Kingdom of Great Britain and Northern I...,GBR,6,"[GBR.3.4_1, GBR.3.13_1, GBR.3.17_1, GBR.3.20_1...",{'A001': {'Label': 'Important in life: Family'...
